In [441]:
import pandas as pd
from functools import reduce
import numpy as np
import math as math
from os import listdir

In [442]:
df1 = pd.read_csv("./csv/events_up_to_01062018.csv")

/home/tito/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [443]:
df2 = pd.read_csv("./csv/labels_training_set.csv")

In [444]:
df1['timestamp'] = pd.to_datetime(df1['timestamp'])

In [445]:
def modelo(obj):
    if(pd.isnull(obj)):
        return obj
    
    list = obj.split(" ")
    if len(list) == 1:
        return obj
    if "iPhone" in list:
        return "iPhone"
    if "Samsung" in list:
        return "Samsung"
    if "Motorola" in list:
        return "Motorola"
    if "Lenovo" in list:
        return "Lenovo"
    if "Sony" in list:
        return "Sony"
    if "LG" in list:
        return "LG"
    if "Asus" in list:
        return "Asus"
    if "iPad" in list:
        return "iPad"
    if "Quantum" in list:
        return "Quantum"
    return obj

df1['model'] = df1['model'].apply(modelo)

In [446]:
df1['model'].value_counts()

iPhone              593603
Samsung             501307
Motorola            162914
LG                   24814
Sony                 15599
Lenovo               11382
iPad                  5663
Asus                  4536
Quantum               1693
Outros TV LED 15         1
Xiaomi Redmi 2           1
Name: model, dtype: int64

In [447]:
df1['event'].value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

# Analisis para los eventos distintos de convercion

In [448]:
df_a = df1

In [449]:
#calculo medianas de los modelos con eventos por persona
df_a_model = df_a.dropna(subset=['model']).loc[:,['person','model']].groupby('person')\
                .apply(lambda x: x['model'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"model_no_convercion"})


df_a_storage = df_a.dropna(subset=['storage']).loc[:,['person','storage']].groupby('person')\
                .apply(lambda x: x['storage'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"storage_no_convercion"})
        
        
df_a_condition = df_a.dropna(subset=['condition']).loc[:,['person','condition']].groupby('person')\
                .apply(lambda x: x['condition'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"condition_no_convercion"})
        
        
df_a_color = df_a.dropna(subset=['color']).loc[:,['person','color']].groupby('person')\
                .apply(lambda x: x['color'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"color_no_convercion"})

df_a_search_engine = df_a.dropna(subset=['search_engine']).loc[:,['person','search_engine']].groupby('person')\
                .apply(lambda x: x['search_engine'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"search_engine"})

In [450]:
df_datos_no_convercion = df_a_model.merge(df_a_storage,how='left',on='person')\
                                .merge(df_a_condition,how='left',on='person')\
                                .merge(df_a_color,how='left',on='person')\
                                .merge(df_a_search_engine,how='left',on='person')

In [451]:
dffinal = df_datos_no_convercion

# Date

# eventos por dia x usr ( al temporario)

In [452]:
# creo columnas con lunes == true o false
def dds(ele):# 0 -> lunes
    if(ele.month == 1):
        return (((ele.day + 6) % 7) == (i))
    if(ele.month == 2):
        return (((ele.day + 3 + 6) % 7) == i)
    if(ele.month == 3):
        return (((ele.day + 3 + 6) % 7) ==(i))
    if(ele.month == 4):
        return (((ele.day + 12) % 7) ==(i))
    if(ele.month == 5):
        return (((ele.day + 1 + 6) % 7) == (i))
i = 0
df1["lunes"] = df1["timestamp"].apply(dds)
i = 1
df1["martes"] = df1["timestamp"].apply(dds)
i = 2
df1["miercoles"] = df1["timestamp"].apply(dds)
i = 3
df1["jueves"] = df1["timestamp"].apply(dds)
i = 4
df1["viernes"] = df1["timestamp"].apply(dds)
i = 5
df1["sabado"] = df1["timestamp"].apply(dds)
i = 6
df1["domingo"] = df1["timestamp"].apply(dds)


In [453]:
dffinal = dffinal.set_index("person")

# mayor cantidad de eventos separados por dia de la semaan por usr 

In [454]:
# mas actividad x dia semana
def diaMasGrande(fila):
    aux = fila["lunes"]
    if(aux < fila["martes"]):
        aux = fila["martes"]
    if(aux < fila["miercoles"]):
        aux = fila["miercoles"]
    if(aux < fila["jueves"]):
        aux = fila["jueves"]
    if(aux < fila["viernes"]):
        aux = fila["viernes"]
    if(aux < fila["sabado"]):
        aux = fila["sabado"]
    if(aux < fila["domingo"]):
        aux = fila["domingo"]
    return aux
dffinal["numero mas usado dia"] = df1[["lunes","martes","miercoles","jueves","viernes","sabado","domingo","person"]].groupby("person").sum().apply(diaMasGrande,axis = 1)

# dia de la semana con mas eventos por usr

In [455]:
# dia de la semaan mas activa
def strDiaMasGrande(fila):
    strAux = "lunes"
    aux = fila["lunes"]
    if(aux < fila["martes"]):
        strAux = "martes"
        aux = fila["martes"]
    if(aux < fila["miercoles"]):
        strAux = "miercoles"
        aux = fila["miercoles"]
    if(aux < fila["jueves"]):
        strAux = "jueves"
        aux = fila["jueves"]
    if(aux < fila["viernes"]):
        strAux = "viernes"
        aux = fila["viernes"]
    if(aux < fila["sabado"]):
        strAux = "sabado"
        aux = fila["sabado"]
    if(aux < fila["domingo"]):
        strAux = "domingo"
        aux = fila["domingo"]
    return strAux
dffinal["nombre mas usado dia"] = df1[["lunes","martes","miercoles","jueves","viernes","sabado","domingo","person"]].groupby("person").sum().apply(strDiaMasGrande,axis = 1)

# cuento eventos por persona

In [456]:
aux = df1[["event","person"]]
aux["aux"] = 1
aux = aux.groupby(["person","event"]).count()
aux = aux.unstack().fillna(0).reset_index().reset_index()
aux.columns = aux.columns.droplevel()
aux["person"] = aux.iloc[:, 1].values
aux = aux.drop("",axis = 1)

/home/tito/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [457]:
dffinal = dffinal.reset_index()

In [458]:
aux = pd.merge(aux,dffinal,on="person",how="inner")

In [459]:
aux.columns

Index(['ad campaign hit', 'brand listing', 'checkout', 'conversion',
       'generic listing', 'lead', 'search engine hit', 'searched products',
       'staticpage', 'viewed product', 'visited site', 'person',
       'model_no_convercion', 'storage_no_convercion',
       'condition_no_convercion', 'color_no_convercion', 'search_engine',
       'numero mas usado dia', 'nombre mas usado dia'],
      dtype='object')

In [460]:
aux = aux.dropna(subset=["storage_no_convercion"])

In [461]:
aux["search_engine"] = aux["search_engine"].fillna("not")

In [462]:
aux["storage_no_convercion"] = aux["storage_no_convercion"].str.replace("GB","").replace("512MB","0.5")

# ultimo evento/fecha de actividad/de de la semana activo
ultimos evento y su fecha y dias de la semana

In [463]:
temp = df1[["person","timestamp"]].groupby("person").max().reset_index()
aux["ultima_fecha_de_actividad"] = temp["timestamp"]

In [464]:
diasDeLaSemana =["lunes","martes","miercoles","jueves","viernes","sabado","domingo"]
def dds2(ele):# 0 -> lunes
    if(ele.month == 1):
        for i in range(0,7):
            if(((ele.day + 6) % 7) == (i)):
                return diasDeLaSemana[i]  
    if(ele.month == 2):
        for i in range(0,7):
            if(((ele.day + 9) % 7) == (i)):
                return diasDeLaSemana[i]
    if(ele.month == 3):
        for i in range(0,7):
            if(((ele.day + 9) % 7) == (i)):
                return diasDeLaSemana[i]
    if(ele.month == 4):
        for i in range(0,7):
            if(((ele.day + 12) % 7) == (i)):
                return diasDeLaSemana[i]
    if(ele.month == 5):
        for i in range(0,7):
            if(((ele.day) % 7) == (i)):
                return diasDeLaSemana[i]
            
aux["ultimo_dia_de_la_semana_activo"] = aux["ultima_fecha_de_actividad"].apply(dds2)

In [465]:
temp = df1[["person","timestamp"]].groupby("person")["timestamp"].max().reset_index()

In [466]:
df2 = pd.merge( df1[["person","timestamp","event"]],temp,on=["person","timestamp"],how="right")
#personas con varios eventos en el mismo timestamp/ como elegir cual evento fue el ultimo? 
#-->uso moda x q no se me ocurre nada mejor
tmpmod = df2.groupby("person").agg(pd.Series.mode)

In [467]:
aux["ultimo_evento_registrado_cmode"] = tmpmod.reset_index()["event"]

In [468]:
aux.columns

Index(['ad campaign hit', 'brand listing', 'checkout', 'conversion',
       'generic listing', 'lead', 'search engine hit', 'searched products',
       'staticpage', 'viewed product', 'visited site', 'person',
       'model_no_convercion', 'storage_no_convercion',
       'condition_no_convercion', 'color_no_convercion', 'search_engine',
       'numero mas usado dia', 'nombre mas usado dia',
       'ultima_fecha_de_actividad', 'ultimo_dia_de_la_semana_activo',
       'ultimo_evento_registrado_cmode'],
      dtype='object')

In [469]:
temp = pd.to_datetime("06.01.2018") - aux["ultima_fecha_de_actividad"]

In [470]:
#lo paso a dias hasta el 1 / 06
aux["dias_hasta"] = temp.reset_index()["ultima_fecha_de_actividad"].dt.days

In [471]:
word = df1["event"].unique()
word

array(['viewed product', 'searched products', 'ad campaign hit',
       'staticpage', 'checkout', 'search engine hit', 'conversion',
       'generic listing', 'brand listing', 'visited site', 'lead'],
      dtype=object)

In [472]:
# fecha ultimo evento
ultimosEveFecha = df1[["person","timestamp"]].groupby("person")["timestamp"].max().reset_index().set_index("person")
del ultimosEveFecha["timestamp"]

In [473]:
eventList = df1["event"].unique()
for word in eventList:
    ultimosEveFecha[word + "_last"] = df1.loc[df1["event"] == word][["person","timestamp"]].groupby("person").max()

In [474]:
for word in eventList:
    ultimosEveFecha[word + "_last"] = (pd.to_datetime("06.01.2018") - pd.to_datetime(ultimosEveFecha[word + "_last"])).dt.days
    ultimosEveFecha[word + "_last"] = ultimosEveFecha[word + "_last"].fillna(ultimosEveFecha[word + "_last"].mode()[0])

In [ ]:
#testear diferentes fills : en la linea de arriba cambiar mode() por el modo de relleno

In [475]:

ultimosEveFecha = ultimosEveFecha.reset_index()

In [476]:
aux = pd.merge(aux,ultimosEveFecha,on="person",how="left")

In [477]:
aux.isna().sum()

ad campaign hit                   0
brand listing                     0
checkout                          0
conversion                        0
generic listing                   0
lead                              0
search engine hit                 0
searched products                 0
staticpage                        0
viewed product                    0
visited site                      0
person                            0
model_no_convercion               0
storage_no_convercion             0
condition_no_convercion           0
color_no_convercion               0
search_engine                     0
numero mas usado dia              0
nombre mas usado dia              0
ultima_fecha_de_actividad         0
ultimo_dia_de_la_semana_activo    0
ultimo_evento_registrado_cmode    0
dias_hasta                        6
viewed product_last               0
searched products_last            0
ad campaign hit_last              0
staticpage_last                   0
checkout_last               

In [478]:
#cantidad de dias en los q entro
df1["dias_hasta_1/6"] = (pd.to_datetime("06.01.2018") - df1["timestamp"]).dt.days

In [479]:
aux = aux.set_index("person")
aux["cantidad_de_dias_utilizado"] = df1[["person","dias_hasta_1/6"]].groupby("person")["dias_hasta_1/6"].nunique().reset_index().set_index("person")
aux = aux.reset_index()

In [480]:
eventList

array(['viewed product', 'searched products', 'ad campaign hit',
       'staticpage', 'checkout', 'search engine hit', 'conversion',
       'generic listing', 'brand listing', 'visited site', 'lead'],
      dtype=object)

In [481]:
def concatenador(item):
       return reduce(lambda x, y: x + y, item)

In [482]:
#def enlistar(x):
#    return [x]
#df1["dias_hasta_1/6"] = df1["dias_hasta_1/6"].apply(enlistar)

In [483]:
#aux["fechas_de_conversiones"] = df1[["event","person","dias_hasta_1/6"]].loc[df1["event"] == "conversion"].groupby("person")["dias_hasta_1/6"].agg(concatenador).reset_index()[0]

In [484]:
#aux["fechas_de_busqueda"] = df1[["event","person","dias_hasta_1/6"]].loc[df1["event"] == "searched products"].groupby("person")["dias_hasta_1/6"].agg(concatenador).reset_index()[0]

In [485]:
df1["dia_de_la_semana"]= df1["timestamp"].dt.dayofweek

In [486]:
#aux["media_dia_de_la_semana_convercion"] = df1.loc[df1.event == "conversion",["person","dia_de_la_semana"]].groupby('person')\
 #       .apply(lambda x: x["dia_de_la_semana"].value_counts().idxmax()).reset_index()[0]

In [487]:
aux["media_dia_de_la_semana_no_convercion"] = df1.loc[df1.event != "conversion",["person","dia_de_la_semana"]].groupby('person')\
        .apply(lambda x: x["dia_de_la_semana"].value_counts().idxmax()).reset_index()[0]

In [488]:
temp = df1[["dias_hasta_1/6","person"]].groupby("person").apply(lambda x: x["dias_hasta_1/6"].max() - x["dias_hasta_1/6"].min()).reset_index() 

aux = pd.merge(aux,temp,on="person",how="left").rename(index=str, columns={0: "delta_ultimo_primer_dia_actividad"})

In [489]:
temp = df1[["dias_hasta_1/6","person"]].groupby("person").apply(lambda x: x["dias_hasta_1/6"].max() - x["dias_hasta_1/6"].nlargest(2).values[-1]).reset_index()

aux = pd.merge(aux,temp,on="person",how="left").rename(index=str, columns={0: "delta_ultimo_anteulit_tiempo_actividad"})

In [490]:
temp = df1[["dias_hasta_1/6","person"]].groupby("person").apply(lambda x: x["dias_hasta_1/6"].nsmallest(2).values[-1] - x["dias_hasta_1/6"].min()).reset_index()

aux = pd.merge(aux,temp,on="person",how="left").rename(index=str, columns={0: "delta_primer_seg_tiempo_actividad"})

In [491]:
df1 = df1.sort_values(by=['person', 'dias_hasta_1/6'])


In [492]:

df1["dTimeEveMax"] = df1[["dias_hasta_1/6","person"]].groupby('person').diff().fillna(0)

In [493]:
temp = df1[["person","dTimeEve"]].groupby('person').max().reset_index()
aux = pd.merge(aux,temp,on="person",how="left")

In [496]:
aux.isna().sum()

person                                    0
ad campaign hit                           0
brand listing                             0
checkout                                  0
conversion                                0
generic listing                           0
lead                                      0
search engine hit                         0
searched products                         0
staticpage                                0
viewed product                            0
visited site                              0
model_no_convercion                       0
storage_no_convercion                     0
condition_no_convercion                   0
color_no_convercion                       0
search_engine                             0
numero mas usado dia                      0
nombre mas usado dia                      0
ultimo_dia_de_la_semana_activo            0
ultimo_evento_registrado_cmode            0
dias_hasta                                6
viewed product_last             

In [495]:
del aux["ultima_fecha_de_actividad"]
aux.to_csv("features.csv",index = False)